<a href="https://colab.research.google.com/github/liege1997/Sign-Language-Recognition/blob/main/silar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Silar - Sign Language Recognition

**Initial methods to import dataset from Kaggle**

import os

os.chdir('/content/drive/MyDrive/Colab Notebooks/ProjectSilar')

os.listdir()

!pip install kaggle

!kaggle datasets download -d grassknoted/asl-alphabet

!unzip -q 'american-sign-language-dataset.zip'

**References:**

https://colab.research.google.com/github/xn2333/OpenCV/blob/master/Image_Processing_in_Python_Final.ipynb#scrollTo=6dy-iP-VTibt

https://developers.google.com/machine-learning/practica/image-classification/convolutional-neural-networks

https://www.tensorflow.org/tutorials/load_data/images

https://www.tensorflow.org/tutorials/images/cnn

In [ ]:
import numpy as np 
import pandas as pd 
import cv2 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
img_folder="../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train"
asl_dir = tf.keras.preprocessing.image_dataset_from_directory(img_folder)

Found 25291 files belonging to 36 classes.


In [ ]:
l0={'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,
   'a':10,'b':11,'c':12,'d':13,'e':14,'f':15,'g':16,'h':17,'i':18,'j':19,'k':20,'l':21,'m':22,
   'n':23,'o':24,'p':25,'q':26,'r':27,'s':28,'t':29,'u':30,'v':31,'w':32,'x':33,'y':34,'z':35}

In [ ]:
l={'a':10,'b':11,'c':12,'d':13,'e':14,'f':15,'g':16,'h':17,'i':18,'j':19,'k':20,'l':21,'m':22,
   'n':23,'o':24,'p':25,'q':26,'r':27,'s':28,'t':29,'u':30,'v':31,'w':32,'x':33,'y':34,'z':35}

In [ ]:
img_height = 32
img_width = 32
load_size = 100 #total images per folder
dir_url='../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'

fgbg1 = cv2.createBackgroundSubtractorMOG2()

def create_ds(u,l):
    image_ds=[]
    labels=[]
    for folder_name in l:
        folder_path=os.path.join(u,folder_name.upper())
        for im in os.listdir(folder_path)[:load_size]:
            image=cv2.imread(os.path.join(folder_path,im))
            image=cv2.resize(image,(img_height,img_width))
            #image=fgbg1.apply(image)
            image_ds.append(image)
            labels.append(l[folder_name])
    return image_ds,labels
X,Y=create_ds(dir_url,l)

In [ ]:
X =np.array(X)/255.0 #to range 0-1
X=np.reshape(X,(X.shape[0],img_height,img_width,3))
X = X.astype('float32')
Y=np.array(Y)
X.shape

In [ ]:
plt.figure(figsize=(10,5))
plt.imshow(X[0])

In [ ]:
model_dl = keras.Sequential()
model_dl.add(Conv2D(16,(3,3),activation="relu",input_shape=(img_height,img_width,3)))
model_dl.add(MaxPool2D(2,2))
model_dl.add(Dropout(0.2))
model_dl.add(Conv2D(32,(3,3),activation="relu"))
model_dl.add(MaxPool2D(2,2))
model_dl.add(Dropout(0.2))
model_dl.add(Conv2D(64,(3,3),activation="relu"))
model_dl.add(MaxPool2D(2,2))
model_dl.add(Dropout(0.2))
model_dl.add(Flatten())
model_dl.add(Dense(128,activation="relu"))
model_dl.add(Dropout(0.2))
model_dl.add(Dense(36,activation="softmax"))
model_dl.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)

from keras.callbacks import EarlyStopping,ReduceLROnPlateau #Import callback functions
earlystop=EarlyStopping(patience=10) #Monitor the performance. If it dips, then stop training
learning_rate_reduce=ReduceLROnPlateau(monitor="val_acc",min_lr=0.001) #Change learning rate if not performing good enough
callbacks=[earlystop,learning_rate_reduce]

model_dl.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=callbacks, epochs=50)

In [ ]:
model_dl.save("model_dl.h5")
model_dl = keras.models.load_model("model_dl.h5")

In [ ]:
#We´ll use any image sample from the Kaggle dataset to test it 

from keras.preprocessing import image

#Creating a dictionary to map each of the indexes to the corresponding number or letter

dict = {0:"0",1:"1",2:"2",3:"3",4:"4",5:"5",6:"6",7:"7",8:"8",9:"9",10:"a",11:"b",12:"c",13:"d",14:"e",15:"f",16:"g",
        17:"h",18:"i",19:"j",20:"k",21:"l",22:"m",23:"n",24:"o",25:"p",26:"q",27:"r",28:"s",29:"t",30:"u",31:"v",32:"w",
        33:"x",34:"y",35:"z"}

#Predicting images
img_url="../input/handgestures/WhatsApp Image 2021-04-28 at 9.36.52 PM (1).jpeg"
#img_url="../input/hand-gesture-b/hand.jpeg"
img = image.load_img(img_url, target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
batch_size=32
image = np.vstack([x])
classes = model_dl.predict_classes(image, batch_size=batch_size)
probabilities = model_dl.predict_proba(image, batch_size=batch_size)
probabilities_formatted = list(map("{:.2f}%".format, probabilities[0]*100))

print(classes) #displaying matrix prediction position

print(f'The predicted image corresponds to "{dict[classes.item()]}" with {probabilities_formatted[classes.item()]} probability.') 